In [90]:
# import itertools as it
# from multiprocessing import Pool, cpu_count
# from collections import defaultdict
import re
import numpy as np
from functools import reduce
from operator import mul
from bisect import insort
with open('input.txt', 'r') as file:
    input = file.read()

test_input = """\
Register A: 729
Register B: 0
Register C: 0

Program: 0,1,5,4,3,0
"""

### Part 1

Time to take some notes here...

##### COMBO OPERANDS
- Combo operands 0 through 3 represent literal values 0 through 3.
- Combo operand 4 represents the value of register A.
- Combo operand 5 represents the value of register B.
- Combo operand 6 represents the value of register C.
- Combo operand 7 is reserved and will not appear in valid programs.


##### INSTRUCTIONS`
- The `adv` instruction (opcode 0) performs division. The numerator is the value in the A register. The denominator is found by raising 2 to the power of the instruction's combo operand. (So, an operand of 2 would divide A by 4 (2^2); an operand of 5 would divide A by 2^B.) The result of the division operation is truncated to an integer and then written to the A register.
- The `bxl` instruction (opcode 1) calculates the bitwise XOR of register B and the instruction's literal operand, then stores the result in register B.
- The `bst` instruction (opcode 2) calculates the value of its combo operand modulo 8 (thereby keeping only its lowest 3 bits), then writes that value to the B register.
- The `jnz` instruction (opcode 3) does nothing if the A register is 0. However, if the A register is not zero, it jumps by setting the instruction pointer to the value of its literal operand; if this instruction jumps, the instruction pointer is not increased by 2 after this instruction.
- The `bxc` instruction (opcode 4) calculates the bitwise XOR of register B and register C, then stores the result in register B. (For legacy reasons, this instruction reads an operand but ignores it.)
- The `out` instruction (opcode 5) calculates the value of its combo operand modulo 8, then outputs that value. (If a program outputs multiple values, they are separated by commas.)
- The `bdv` instruction (opcode 6) works exactly like the adv instruction except that the result is stored in the B register. (The numerator is still read from the A register.)
- The `cdv` instruction (opcode 7) works exactly like the adv instruction except that the result is stored in the C register. (The numerator is still read from the A register.)


In [104]:
class ChronospatialComputer:
    def __init__(self,txt):
        registers_match = re.findall(r'Register ([ABC]): (\d+)', txt)
        self.registers = {key: int(value) for key, value in registers_match}
        print(self.registers)
        program_match = re.search(r'Program: ([\d,]+)', txt)
        self.program = [int(x) for x in program_match.group(1).split(',')] if program_match else []
        print(self.program)

    def combo(self,op):
        if op < 4:
            return op
        elif op == 4:
            return self.registers['A']
        elif op == 5:
            return self.registers['B']
        elif op == 6:
            return self.registers['C']
        else:
            return 7

    def run_instruction(self,program):
        # combo operands: 0,1,2,3,A,B,C,7
        i = 0
        output = []
        while(i<len(program)):# for i in range(0,len(program),2):
            inst = program[i]
            op = program[i+1]
            # print(f"A:{self.registers['A']}, B:{self.registers['B']}, C:{self.registers['C']}")
            # print(f"inst:{inst} op:{op}")
            match inst:
                case 0: # A / 2^comb_op. truncate res to int. write to A
                    res = self.registers['A'] // (2**self.combo(op))
                    self.registers['A'] = res
                    # print(self.combo(op))
                    # print(f"A:{self.registers['A']}")
                case 1: # bitwise xor with B and lit_op, write to B
                    res = self.registers['B'] ^ op
                    self.registers['B'] = res
                    # print(f"B:{self.registers['B']}")
                case 2: # comb_op % 8, write to B
                    res = self.combo(op) % 8
                    self.registers['B'] = res
                    # print(f"B:{self.registers['B']}")
                case 3: # if A == 0: nothing. else: pointer jumps to lit_op. does not jump 2 after
                    if self.registers['A'] != 0:
                        i = op
                        # jumped = True
                        # print("pointer changed: i={i}")
                        continue
                case 4: # bitwise XOR of B and C. ignores op, stores res in B
                    res = self.registers['B'] ^ self.registers['C']
                    self.registers['B'] = res
                    # print(f"B:{self.registers['B']}")
                case 5: # combo_op % 8, output separated by commas
                    res = self.combo(op) % 8
                    output.append(res)
                    # print(f"output:{output}")
                case 6: # case 0 but writes to B
                    res = self.registers['A'] // (2**self.combo(op))
                    self.registers['B'] = res
                    # print(f"B:{self.registers['B']}")
                case 7: # case 0 but writes to C
                    res = self.registers['A'] // (2**self.combo(op))
                    self.registers['C'] = res
                    # print(f"C:{self.registers['C']}")
            i += 2
        return output


res = ChronospatialComputer(input)
# res.registers['C'] = 9
# res.run_instructio [2,6])
results = res.run_instruction(res.program)
''.join([str(x) + ',' for x in results])

{'A': 59590048, 'B': 0, 'C': 0}
[2, 4, 1, 5, 7, 5, 0, 3, 1, 6, 4, 3, 5, 5, 3, 0]


'6,5,7,4,5,7,3,1,0,'

In [105]:
# Test cases
test = ChronospatialComputer(test_input)

test.registers['C'] = 9
test.run_instruction([2,6])
assert test.registers['B'] == 1

test.registers['A'] = 10
ans = test.run_instruction([5,0,5,1,5,4])
assert ans == [0,1,2]

test.registers['A'] = 2024
ans = test.run_instruction([0,1,5,4,3,0])
assert ans == [4,2,5,6,7,7,7,7,3,1,0]
assert test.registers['A'] == 0

test.registers['B'] = 29
ans = test.run_instruction([1,7])
assert test.registers['B'] == 26
# assert ans == [4,2,5,6,7,7,7,7,3,1,0]

test.registers['B'] = 2024
test.registers['C'] = 43690
ans = test.run_instruction([4,0])
assert test.registers['B'] == 44354

{'A': 729, 'B': 0, 'C': 0}
[0, 1, 5, 4, 3, 0]


There were two major mistakes in my code...
1. used `^` for power instead of `**`. This was disappointing.
2. not realizing that the output is supposed to be literally joined by commas (I assumed like past puzzles that it always takes a number...) I had to ran someone else's answer to see what the correct answer is before getting the first star. Not proud of it.

### Part 2


In [99]:
import sys
from heapq import *

ans = 0

"""
adv - 0 - A / combo() -> A
bxl - 1 - B ^ lit() -> B
bst - 2 - combo() % 8 -> B
jnz - 3 - if A != 0 { jmp literal() }
bxc - 4 - B ^ C -> B (ignore operand)
out - 5 - output combo()
bdv - 6 - A / combo() -> B
cdv - 7 - A / combo() -> C
"""

st = False
registers = []
program = None

file = open('input.txt')
for line in file.readlines():
    if line.strip() == "":
        st = True
        continue
    if not st:
        registers.append(int(line.split(": ")[1]))
    else:
        program = list(map(int,line.split(": ")[1].split(",")))

def interp_combo(regs, val):
    if val < 4: return val
    return regs[val - 4] if val < 7 else None
        
def run_program(regs, prog):
    outt = []
    inst_ptr = 0
    while inst_ptr < len(prog):
        print(inst_ptr, regs)
        code = prog[inst_ptr]
        op = prog[inst_ptr + 1]
        combo_op = interp_combo(regs, op)
        jumped = False

        if code == 0:
            regs[0] = regs[0] // (2**combo_op)
        elif code == 1:
            regs[1] = regs[1] ^ op
        elif code == 2:
            regs[1] = combo_op % 8
        elif code == 3:
            if regs[0] != 0:
                inst_ptr = op
                jumped = True
        elif code == 4:
            regs[1] = regs[1] ^ regs[2]
        elif code == 5:
            outt.append(combo_op % 8)
        elif code == 6:
            regs[1] = regs[0] // (2**combo_op)
        elif code == 7:
            print(op, combo_op)
            regs[2] = regs[0] // (2**combo_op)
        if not jumped:
            inst_ptr += 2
    return outt

print(",".join(map(str,run_program(registers, program))))

0 [59590048, 0, 0]
2 [59590048, 0, 0]
4 [59590048, 5, 0]
5 5
6 [59590048, 5, 1862189]
8 [7448756, 5, 1862189]
10 [7448756, 3, 1862189]
12 [7448756, 1862190, 1862189]
14 [7448756, 1862190, 1862189]
0 [7448756, 1862190, 1862189]
2 [7448756, 4, 1862189]
4 [7448756, 1, 1862189]
5 1
6 [7448756, 1, 3724378]
8 [931094, 1, 3724378]
10 [931094, 7, 3724378]
12 [931094, 3724381, 3724378]
14 [931094, 3724381, 3724378]
0 [931094, 3724381, 3724378]
2 [931094, 6, 3724378]
4 [931094, 3, 3724378]
5 3
6 [931094, 3, 116386]
8 [116386, 3, 116386]
10 [116386, 5, 116386]
12 [116386, 116391, 116386]
14 [116386, 116391, 116386]
0 [116386, 116391, 116386]
2 [116386, 2, 116386]
4 [116386, 7, 116386]
5 7
6 [116386, 7, 909]
8 [14548, 7, 909]
10 [14548, 1, 909]
12 [14548, 908, 909]
14 [14548, 908, 909]
0 [14548, 908, 909]
2 [14548, 4, 909]
4 [14548, 1, 909]
5 1
6 [14548, 1, 7274]
8 [1818, 1, 7274]
10 [1818, 7, 7274]
12 [1818, 7277, 7274]
14 [1818, 7277, 7274]
0 [1818, 7277, 7274]
2 [1818, 2, 7274]
4 [1818, 7, 7274